# Comics Rx
## [A comic book recommendation system](https://github.com/MangrobanGit/comics_rx)
<img src="https://images.unsplash.com/photo-1514329926535-7f6dbfbfb114?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjEyMDd9&auto=format&fit=crop&w=2850&q=80" width="400" align='left'>

---

# Reduced Data: Grid Search + Cross-Validation

This time, as explored in the EDA NB, let's consider removing customers who we feel have too few or too many purchases to influence the model in the intended way.

Examples:
- Too few - Customers who have only bought 1 comic (series).
- Too many - Customers with > 1000 series (for example, think all eBay customers are rolled into one account number).

# Libraries

In [4]:
%matplotlib inline
%load_ext autoreload
# %autoreload 1 #would be where you need to specify the files
# %aimport comic_recs

import pandas as pd # dataframes
import os
import pickle
import numpy as np

# Data storage
from sqlalchemy import create_engine # SQL helper
import psycopg2 as psql #PostgreSQL DBs

# import Spark methods
import pyspark
from pyspark.sql import SparkSession
from pyspark.ml.evaluation import RegressionEvaluator
# from pyspark.sql.types import (StructType, StructField, IntegerType
#                                ,FloatType, LongType, StringType)
from pyspark.sql.types import *

import pyspark.sql.functions as F
from pyspark.sql.functions import col, explode, lit, isnan, when, count
from pyspark.ml.recommendation import ALS, ALSModel
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import (CrossValidator, ParamGridBuilder, 
                               TrainValidationSplit)
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.sql import DataFrame
from pyspark import SparkConf
from pyspark.sql import SQLContext

# Plotting
import seaborn as sns
import matplotlib.pyplot as plt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import sys

In [6]:
sys.path.append('..')

In [7]:
# Custom
import data_fcns as dfc
import keys  # Custom keys lib
import comic_recs as cr

import time
import itertools
from functools import reduce
import numpy as np

### Configure Spark

In [8]:
conf = SparkConf()

# Laptop config
# conf = (conf.setMaster('local[*]')
# #         .set('spark.executor.memory', '1G') #https://stackoverflow.com/questions/48523629/spark-pyspark-an-error-occurred-while-trying-to-connect-to-the-java-server-127
#         .set('spark.driver.memory', '7G')
#         .set('spark.driver.maxResultSize', '2G'))
# #         .set('spark.executor.memory', '1G')
# #         .set('spark.driver.memory', '10G')
# #         .set('spark.driver.maxResultSize', '5G'))

# AWS instance config
conf = (conf.setMaster('local[*]')
        .set('spark.driver.memory', '25G')
        .set('spark.driver.maxResultSize', '4G'))
#         .set('spark.executor.memory', '1G') #https://stackoverflow.com/questions/48523629/spark-pyspark-an-error-occurred-while-trying-to-connect-to-the-java-server-127
#         .set('spark.driver.memory', '10G')
#         .set('spark.driver.maxResultSize', '5G')
#         )

sc = pyspark.SparkContext().getOrCreate(conf=conf)

sql_context = SQLContext(sc)

sc.setCheckpointDir('./checkpoints')
# spark.sparkContext.setCheckpointDir("hdfs://datalake/check_point_directory/als")

## Import Data

We've previously set aside the dataset into a `json` file.

In [9]:
# We have previously created a version of the transactions table 
# and filtered it down.
sold = sql_context.read.json('raw_data/als_input_filtered_190915.json')

In [10]:
# Persist the data
sold.persist()

DataFrame[account_id: bigint, bought: bigint, comic_id: bigint]

In [11]:
sold.count()

846090

### ALS Model

Let's start with  train/test split.

In [12]:
random_seed = 41916

In [13]:
# Split data into training and test set
(train, test) = sold.randomSplit([.75, .25], seed=random_seed)

In [14]:
train.count()

634635

Make sure shapes make sense.

In [15]:
print(train.count(), len(train.columns))

634635 3


In [16]:
print(test.count(), len(test.columns))

211455 3


## New Grid Search

In [14]:
# # hyper-param config
# max_iters = [20, 25, 30]
# ranks = [5, 10, 20, 25]
# reg_params = [.05, .10, .15]
# alphas = [5, 25, 40]

In [15]:
# # hyper-param config
# max_iters = [30, 40]
# ranks = [25, 30]
# reg_params = [.15, .25]
# alphas = [5]

In [16]:
# # hyper-param config
# max_iters = [30]
# ranks = [5, 10]
# reg_params = [.25, .30]
# alphas = [5]

In [17]:
# # hyper-param config
# max_iters = [40]
# ranks = [30]
# reg_params = [.25]
# alphas = [5]

In [24]:
# hyper-param config
max_iters = [20, 30, 40]
ranks = [5, 20, 40, 75, 90]
reg_params = [.01, .05, .25, .50]
alphas = [5, 25, 40]

These will be user, item and rating columns, respectively, for our dataset.
- `account_id`
- `comic_id`
- `bought`

In [25]:
model_list = cr.create_als_models_list(userCol="account_id", itemCol="comic_id"
                                    ,ratingCol="bought"
                                    ,ranks=ranks
                                    ,max_iters=max_iters
                                    ,reg_params=reg_params
                                    ,alphas=alphas
                                    ,seed=random_seed
                                   )

In [26]:
model_list

[ALS_e3a5f70111f2,
 ALS_31dc9c1bea61,
 ALS_694004ec4b4b,
 ALS_ed72acb68625,
 ALS_ae4b80e5ecc7,
 ALS_1e0c52379461,
 ALS_b4897fb196e8,
 ALS_80779c63492d,
 ALS_2fb14632d6a5,
 ALS_19a02ec671f5,
 ALS_c6ebc286ba0e,
 ALS_972979b7c85e,
 ALS_425396e875b3,
 ALS_452fdf3dc88c,
 ALS_48e163559d45,
 ALS_075d19fb6a15,
 ALS_875880ca3008,
 ALS_8a9498c65b20,
 ALS_60feabebffc0,
 ALS_45c17e2ba59c,
 ALS_ca7726e251d6,
 ALS_ba583b9fa9c8,
 ALS_836eb445a7cb,
 ALS_514b35eb5433,
 ALS_5f656bdb60f0,
 ALS_819ac2c567e0,
 ALS_94c735e565c9,
 ALS_432ba0124b90,
 ALS_f74aa46e7340,
 ALS_69095198ccef,
 ALS_9f58bb37d8a6,
 ALS_0175224aa5e7,
 ALS_d547ac6327bf,
 ALS_f2abed616a70,
 ALS_13e297914a1b,
 ALS_1d7d650d0b2c,
 ALS_1a077cc9524a,
 ALS_dcbaa2160472,
 ALS_9b9fbbf9f4e1,
 ALS_e929417bef58,
 ALS_22fe5f9efc4e,
 ALS_7de90eeb71f5,
 ALS_fdf1d5902e0e,
 ALS_0768543d717a,
 ALS_bcafc5a2400b,
 ALS_ef9b5521eb47,
 ALS_ec324f68d299,
 ALS_fce2a4cf2405,
 ALS_97213ba87c2b,
 ALS_fc3924fd668c,
 ALS_79c83c34e408,
 ALS_bc44f23ee90e,
 ALS_8b0381b

Check that model list has proper number of combinations.

In [27]:
len(model_list)

180

In [22]:
print( len(model_list) == (len(max_iters)*len(ranks)*len(reg_params)*len(alphas)) )

True


### Manual Cross-Validation

## Metrics

In [23]:
ROEMs = cr.get_ROEMs(sql_context, model_list, train, test, 'account_id'
                     ,'bought')

Validation ROEM #1: 0.19869719550926318
Validation ROEM #2: 0.19433399830675785
Validation ROEM #3: 0.19657179442552847
Validation ROEM #4: 0.19727919618163878
Validation ROEM #5: 0.19135726332450673
Validation ROEM #6: 0.19600523404055623
Validation ROEM #7: 0.19613417381855794
Validation ROEM #8: 0.1883132407551012
Validation ROEM #9: 0.19068610498273622
Validation ROEM #10: 0.19615494652631793
Validation ROEM #11: 0.18812054171315873
Validation ROEM #12: 0.19001151310123257
Validation ROEM #13: 0.19825515409629393
Validation ROEM #14: 0.19304004913991987
Validation ROEM #15: 0.1965050670149716
Validation ROEM #16: 0.1970328332908752
Validation ROEM #17: 0.18878406923020186
Validation ROEM #18: 0.19591794813692
Validation ROEM #19: 0.1959248359391219
Validation ROEM #20: 0.18704623543343596
Validation ROEM #21: 0.18922641743172072
Validation ROEM #22: 0.19526198351646434
Validation ROEM #23: 0.1863824686096206
Validation ROEM #24: 0.18846383383383283
Validation ROEM #25: 0.1980254962

In [24]:
best_model = cr.get_best_model(ROEMs, model_list)

Index of smallest error: 177
Smallest error:  0.1383817704950063


In [25]:
best_model

ALS_811ff35a261b

## Extracting Parameters

In [26]:
# Extract the best_modelel parameters
print('alpha: {}'.format(best_model.getAlpha()))
print('reg param: {}'.format(best_model.getRegParam()))
print('max iter: {}'.format(best_model.getMaxIter()))
print('rank: {}'.format(best_model.getRank()))

alpha: 5.0
reg param: 0.5
max iter: 40
rank: 90


In [31]:
mod_alphas = [mod.getAlpha() for mod in model_list]
mod_reg_params = [mod.getRegParam() for mod in model_list]
mod_ranks = [mod.getRank() for mod in model_list]
mod_max_iters = [mod.getMaxIter() for mod in model_list]


Upon inspection of the ROEM history, it appears that the best model lies along the edge of our grid, so we could potentially try other parameter values beyond. For example, the best model has a rank of 40, but we only tried rank values up to 40. We could potentially try higher values. 

### Assemble the model metrics and error metrics into pandas dataframe

In [ ]:
with open('support_data/roems_20190922.pkl', 'wb') as f:
    pickle.dump(ROEMs, f)
    

In [17]:
pickle_in = open("support_data/roems_20190922.pkl","rb")
ROEMSs_load = pickle.load(pickle_in)

In [32]:
model_dict = {'alpha': mod_alphas
              ,'reg_param': mod_reg_params
              ,'rank': mod_ranks
              ,'max_iter': mod_max_iters
              ,'ROEM': ROEMs
}

In [64]:
model_df = pd.DataFrame(model_dict)

In [65]:
model_df = model_df.sort_values(by=['ROEM'])

In [66]:
model_df['enumer'] = list(range(1,len(model_df)+1))

In [68]:
model_df.head(10)

,alpha,reg_param,rank,max_iter,ROEM,enumer
177,5.0,0.50,90,40,0.138382,1
165,5.0,0.50,90,30,0.138528,2
129,5.0,0.50,75,30,0.139069,3
141,5.0,0.50,75,40,0.139095,4
153,5.0,0.50,90,20,0.139222,5
117,5.0,0.50,75,20,0.139877,6
138,5.0,0.25,75,40,0.144694,7
105,5.0,0.50,40,40,0.144929,8
93,5.0,0.50,40,30,0.145185,9
126,5.0,0.25,75,30,0.145319,10


The top 6 models appears to pretty tightly clustered in ROEM. In this case I will opt to use the model that's not on the edge of the grid. I think index 129 will do.

In [70]:
best_model = model_list[129]

In [71]:
# Extract the best_modelel parameters
print('alpha: {}'.format(best_model.getAlpha()))
print('reg param: {}'.format(best_model.getRegParam()))
print('max iter: {}'.format(best_model.getMaxIter()))
print('rank: {}'.format(best_model.getRank()))

alpha: 5.0
reg param: 0.5
max iter: 30
rank: 75


#### Save the best model

In [72]:
!rm -rf models/best_model_use_20190922/

In [73]:
best_model.save('models/best_model_use_20190922')

## Cross-Validation

Perform 5-fold cross-validation

In [75]:
errors = []

In [76]:
foldlist = cr.create_spark_5_fold_set(train, seed=random_seed)

In [77]:
errors = cr.perform_cv(foldlist, best_model, sql_context, 
                       'account_id', 'bought')

In [78]:
errors

[0.152455197544284,
 0.1596836579518168,
 0.15677413940453713,
 0.15669430633872505,
 0.1517828824930433]

In [91]:
print("CV errors range: %0.3f (+/- %0.3f)" % (np.mean(errors), 
                                              np.std(errors) * 2))

CV errors range: 0.155 (+/- 0.006)


Seems pretty stable, which helps alleviate some over-fitting concerns.

## Test the Candidate Model

Test vs our holdout set.

In [80]:
model_fitted = best_model.fit(train)

In [81]:
# get predictions on test
test_preds = model_fitted.transform(test)

# Evaluate test
test_roem = cr.calculate_ROEM(sql_context, test_preds, 'account_id', 'bought')
test_roem

0.13845788390498442

Weird that test error is lower than train...but not too far off. Let's run with it for now.

## Get Factors

#### Save the item factors for future use!

In [82]:
item_factors = model_fitted.itemFactors.toPandas()

In [83]:
item_factors.shape

(790, 2)

In [84]:
item_factors.to_pickle("support_data/item_factors_20190922.pkl")

In [85]:
pd.set_option('display.max_colwidth', -1)

In [86]:
item_factors.head()

,id,features
0,60,"[0.0, 0.0, 0.0, 0.0826452448964119, 0.0, 0.0, 0.0, 0.0, 0.039434973150491714, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.9948146939277649, 0.0, 0.0, 0.2473047822713852, 0.02167966403067112, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.059246089309453964, 0.03268354758620262, 0.0, 0.10995704680681229, 0.0, 0.07982633262872696, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.015057926997542381, 0.011894325725734234, 0.0, 0.0, 0.0, 0.0, 0.0, 0.06303215771913528, 0.0, 0.0, 0.0, 0.0, 0.0, 0.033097922801971436, 0.031103042885661125, 0.028060728684067726, 0.01773754693567753, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.19910746812820435, 0.0]"
1,80,"[0.0, 0.0, 0.0, 0.0033040973357856274, 0.0, 0.0, 0.0, 0.0, 0.009639354422688484, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.01085046399384737, 0.0, 0.0, 0.0, 0.0, 0.0, 0.4813947081565857, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.024865329265594482, 0.0, 0.0, 0.0, 0.0, 0.33433255553245544, 0.14560888707637787, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.6264654994010925, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1923040896654129, 0.0, 0.0, 0.06339741498231888, 0.0, 0.0, 0.005075078923255205, 0.0, 0.0, 0.0, 0.0, 0.0, 0.128806933760643, 0.0, 0.0, 0.05120192840695381, 0.0, 0.0, 0.02547549456357956, 0.0]"
2,110,"[0.0, 0.009681249968707561, 0.3157486915588379, 0.035485029220581055, 0.08428007364273071, 0.05139074847102165, 0.010632488876581192, 0.010384040884673595, 0.0, 0.004306851886212826, 0.1360526829957962, 0.0, 0.0, 0.0, 0.0, 0.11445901542901993, 0.0, 0.02609862945973873, 0.08677253127098083, 0.0, 0.1700509786605835, 0.01852649636566639, 0.01742601953446865, 0.0, 0.052328720688819885, 0.0, 0.08754375576972961, 0.0, 0.0, 0.05530954897403717, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.10496184229850769, 0.022573139518499374, 0.0, 0.06583559513092041, 0.09494148939847946, 0.0, 0.0831538662314415, 0.0, 0.03067944385111332, 0.036219604313373566, 0.016076000407338142, 0.07238824665546417, 0.0, 0.031701963394880295, 0.012026206590235233, 0.0, 0.0, 0.0, 0.0, 0.0, 0.06698817759752274, 0.3719470500946045, 0.0, 0.0, 0.03545656055212021, 0.0, 0.0, 0.0, 0.0, 0.0, 0.06538882851600647, 0.10146535187959671, 1.110505223274231, 0.0, 0.07677265256643295, 0.0401727668941021, 0.028192708268761635, 0.0, 0.0]"
3,200,"[0.0, 0.0, 0.0008219783194363117, 0.05931999906897545, 0.0017522152047604322, 0.060892585664987564, 0.0, 0.5071122050285339, 0.1063532903790474, 0.005306006409227848, 0.005510434973984957, 0.0, 0.014103081077337265, 0.0, 0.0, 0.0897526741027832, 0.1523221731185913, 0.0, 0.11788218468427658, 0.0, 0.0, 0.10728922486305237, 0.010851364582777023, 0.011304203420877457, 0.0, 0.0, 0.027045346796512604, 0.02051759511232376, 0.11239876598119736, 0.004266583826392889, 0.06424926221370697, 0.011516495607793331, 0.06823383271694183, 0.0, 0.0, 0.07253396511077881, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1502048820257187, 0.10489537566900253, 0.09956609457731247, 0.0, 0.0, 0.010047723539173603, 0.08179843425750732, 0.0, 0.13192865252494812, 0.016740024089813232, 0.0, 0.07769604027271271, 0.1828979104757309, 0.0, 0.0, 0.0, 0.0, 0.2341715395450592, 0.002455874113366008, 0.5527805685997009, 0.03604438528418541, 0.11876046657562256, 0.0, 0.0, 0.584153950214386, 0.0851968452334404, 0.03369034081697464, 0.06624818593263626, 0.0, 0.0, 0.05054672062397003, 0.05392900109291077, 0.0]"
4,240,"[0.0, 0.2235216647386551, 0.0, 0.0, 0.0, 0.0, 0.0, 0.07523377239704132, 0.15706036984920502, 0.02980225533246994, 0.0, 0.0, 0.0, 0.03824526444077492, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.17709434032440186, 0.0, 0.0, 0.098581463098526, 0.0, 0.1592487245798111, 0.0, 0.1055234745144844, 0.059581972658634186, 0.0, 0.1497793197631836, 0.07032154500484467, 0.0, 0.10109743475914001, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.11361673474311829, 0.0, 0.0, 0.0, 0.09224706143140793, 0.2049531638622284, 0.0, 0.0, 0.0, 0.11741948872804642, 0.0, 0.0004655775846913457, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.042575541883707047, 0.0,

## Test Retrieving the Factors

In [87]:
unpickled_items = pd.read_pickle('support_data/item_factors_20190922.pkl')

In [88]:
unpickled_items.head()

,id,features
0,60,"[0.0, 0.0, 0.0, 0.0826452448964119, 0.0, 0.0, 0.0, 0.0, 0.039434973150491714, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.9948146939277649, 0.0, 0.0, 0.2473047822713852, 0.02167966403067112, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.059246089309453964, 0.03268354758620262, 0.0, 0.10995704680681229, 0.0, 0.07982633262872696, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.015057926997542381, 0.011894325725734234, 0.0, 0.0, 0.0, 0.0, 0.0, 0.06303215771913528, 0.0, 0.0, 0.0, 0.0, 0.0, 0.033097922801971436, 0.031103042885661125, 0.028060728684067726, 0.01773754693567753, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.19910746812820435, 0.0]"
1,80,"[0.0, 0.0, 0.0, 0.0033040973357856274, 0.0, 0.0, 0.0, 0.0, 0.009639354422688484, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.01085046399384737, 0.0, 0.0, 0.0, 0.0, 0.0, 0.4813947081565857, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.024865329265594482, 0.0, 0.0, 0.0, 0.0, 0.33433255553245544, 0.14560888707637787, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.6264654994010925, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1923040896654129, 0.0, 0.0, 0.06339741498231888, 0.0, 0.0, 0.005075078923255205, 0.0, 0.0, 0.0, 0.0, 0.0, 0.128806933760643, 0.0, 0.0, 0.05120192840695381, 0.0, 0.0, 0.02547549456357956, 0.0]"
2,110,"[0.0, 0.009681249968707561, 0.3157486915588379, 0.035485029220581055, 0.08428007364273071, 0.05139074847102165, 0.010632488876581192, 0.010384040884673595, 0.0, 0.004306851886212826, 0.1360526829957962, 0.0, 0.0, 0.0, 0.0, 0.11445901542901993, 0.0, 0.02609862945973873, 0.08677253127098083, 0.0, 0.1700509786605835, 0.01852649636566639, 0.01742601953446865, 0.0, 0.052328720688819885, 0.0, 0.08754375576972961, 0.0, 0.0, 0.05530954897403717, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.10496184229850769, 0.022573139518499374, 0.0, 0.06583559513092041, 0.09494148939847946, 0.0, 0.0831538662314415, 0.0, 0.03067944385111332, 0.036219604313373566, 0.016076000407338142, 0.07238824665546417, 0.0, 0.031701963394880295, 0.012026206590235233, 0.0, 0.0, 0.0, 0.0, 0.0, 0.06698817759752274, 0.3719470500946045, 0.0, 0.0, 0.03545656055212021, 0.0, 0.0, 0.0, 0.0, 0.0, 0.06538882851600647, 0.10146535187959671, 1.110505223274231, 0.0, 0.07677265256643295, 0.0401727668941021, 0.028192708268761635, 0.0, 0.0]"
3,200,"[0.0, 0.0, 0.0008219783194363117, 0.05931999906897545, 0.0017522152047604322, 0.060892585664987564, 0.0, 0.5071122050285339, 0.1063532903790474, 0.005306006409227848, 0.005510434973984957, 0.0, 0.014103081077337265, 0.0, 0.0, 0.0897526741027832, 0.1523221731185913, 0.0, 0.11788218468427658, 0.0, 0.0, 0.10728922486305237, 0.010851364582777023, 0.011304203420877457, 0.0, 0.0, 0.027045346796512604, 0.02051759511232376, 0.11239876598119736, 0.004266583826392889, 0.06424926221370697, 0.011516495607793331, 0.06823383271694183, 0.0, 0.0, 0.07253396511077881, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1502048820257187, 0.10489537566900253, 0.09956609457731247, 0.0, 0.0, 0.010047723539173603, 0.08179843425750732, 0.0, 0.13192865252494812, 0.016740024089813232, 0.0, 0.07769604027271271, 0.1828979104757309, 0.0, 0.0, 0.0, 0.0, 0.2341715395450592, 0.002455874113366008, 0.5527805685997009, 0.03604438528418541, 0.11876046657562256, 0.0, 0.0, 0.584153950214386, 0.0851968452334404, 0.03369034081697464, 0.06624818593263626, 0.0, 0.0, 0.05054672062397003, 0.05392900109291077, 0.0]"
4,240,"[0.0, 0.2235216647386551, 0.0, 0.0, 0.0, 0.0, 0.0, 0.07523377239704132, 0.15706036984920502, 0.02980225533246994, 0.0, 0.0, 0.0, 0.03824526444077492, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.17709434032440186, 0.0, 0.0, 0.098581463098526, 0.0, 0.1592487245798111, 0.0, 0.1055234745144844, 0.059581972658634186, 0.0, 0.1497793197631836, 0.07032154500484467, 0.0, 0.10109743475914001, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.11361673474311829, 0.0, 0.0, 0.0, 0.09224706143140793, 0.2049531638622284, 0.0, 0.0, 0.0, 0.11741948872804642, 0.0, 0.0004655775846913457, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.042575541883707047, 0.0,

---